<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim

# **Desafío 2**

### Alumno **Daniel Fernando Herrera** (a1621)

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [275]:
#!pip install gensim
#!pip install keras
# reiniciar la sesion

In [276]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [277]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('/content/vieja_al_centro_de_la_tierra.txt', sep='/n', header=None)
df.head()

<ipython-input-277-2c9b0d0c192f>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,Capítulo 1
1,"El domingo 24 de mayo de 1863, mi tío, el prof..."
2,"Marta, su excelente criada, se preocupó sobrem..."
3,"“Bueno” —pensé para mí—, “si mi tío viene con ..."
4,—¡Tan temprano y ya está aquí el señor Lidenbr...


In [278]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2311


### 1 - Preprocesamiento

In [279]:
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))

def limpiar_y_tokenizar(texto):
    tokens = text_to_word_sequence(texto)
    return [w for w in tokens if w not in stop_words]

sentence_tokens = []
for _, row in df.iterrows():
    sentence_tokens.append(limpiar_y_tokenizar(row[0]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [280]:
# Demos un vistazo
sentence_tokens[:2]

[['capítulo', '1'],
 ['domingo',
  '24',
  'mayo',
  '1863',
  'tío',
  'profesor',
  'lidenbrock',
  'entró',
  'rápidamente',
  'hogar',
  'situado',
  'número',
  '19',
  'könig‑strasse',
  'calles',
  'tradicionales',
  'barrio',
  'antiguo',
  'hamburgo']]

In [281]:
#from keras.preprocessing.text import text_to_word_sequence
#from tensorflow.keras.preprocessing.text import text_to_word_sequence

#sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
#for _, row in df[:None].iterrows():
#    sentence_tokens.append(text_to_word_sequence(row[0]))

In [282]:
# Demos un vistazo
#sentence_tokens[:2]

### 2 - Crear los vectores (word2vec)

In [283]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [284]:
# otros hiperparametros
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    #5 frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       #2 cant de palabras antes y desp de la predicha
                     vector_size=300, #300 dimensionalidad de los vectores
                     negative=20,    #20 cantidad de negative samples... 0 es no se usa
                     workers=1,      #1 si tienen más cores pueden cambiar este valor
                     sg=1)           #1 modelo 0:CBOW  1:skipgram

In [285]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [286]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2311


In [287]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1590


### 3 - Entrenar embeddings

In [288]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 534459.1875
Loss after epoch 1: 239922.4375
Loss after epoch 2: 239333.25
Loss after epoch 3: 208892.875
Loss after epoch 4: 203620.5
Loss after epoch 5: 201522.375
Loss after epoch 6: 201740.0
Loss after epoch 7: 201824.125
Loss after epoch 8: 181502.0
Loss after epoch 9: 170394.5
Loss after epoch 10: 170315.0
Loss after epoch 11: 169068.25
Loss after epoch 12: 167957.0
Loss after epoch 13: 167368.75
Loss after epoch 14: 166110.75
Loss after epoch 15: 164520.5
Loss after epoch 16: 164335.0
Loss after epoch 17: 163029.5
Loss after epoch 18: 162011.5
Loss after epoch 19: 159824.5
Loss after epoch 20: 158947.5
Loss after epoch 21: 152713.5
Loss after epoch 22: 151270.0
Loss after epoch 23: 151756.0
Loss after epoch 24: 149936.5
Loss after epoch 25: 149743.5
Loss after epoch 26: 148718.5
Loss after epoch 27: 148443.0
Loss after epoch 28: 148415.5
Loss after epoch 29: 147726.0


(657129, 1134840)

### 4 - Ensayar

In [289]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["terrestre"], topn=10)

[('mineral', 0.8710821866989136),
 ('sinuosidades', 0.7647475600242615),
 ('gases', 0.7615578174591064),
 ('interiores', 0.7610321640968323),
 ('hulla', 0.7534886002540588),
 ('aumento', 0.7461691498756409),
 ('grietas', 0.7346855998039246),
 ('límite', 0.728818416595459),
 ('suponer', 0.728774905204773),
 ('base', 0.7269209027290344)]

In [290]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["terrestre"], topn=10)

[('tío', -0.1502103954553604),
 ('marcha', -0.16049982607364655),
 ('señor', -0.16068123281002045),
 ('mañana', -0.17053981125354767),
 ('cazador', -0.17226693034172058),
 ('cabeza', -0.17972882091999054),
 ('guía', -0.19161376357078552),
 ('fridriksson', -0.19311699271202087),
 ('—preguntó', -0.19668136537075043),
 ('—no', -0.19725878536701202)]

In [291]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["roca"], topn=10)

[('atracción', 0.7249952554702759),
 ('pozo', 0.7233765125274658),
 ('bóveda', 0.7227107882499695),
 ('existía', 0.7200817465782166),
 ('abierto', 0.7100129127502441),
 ('corriendo', 0.7063911557197571),
 ('sorprendente', 0.7040880918502808),
 ('dedo', 0.7040205597877502),
 ('saliente', 0.7024168372154236),
 ('trozos', 0.6967659592628479)]

In [292]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["roca"])

[('—me', -0.09329775720834732),
 ('—le', -0.10894450545310974),
 ('mañana', -0.12993252277374268),
 ('siguiente', -0.13532446324825287),
 ('quiere', -0.13949473202228546),
 ('¿no', -0.1418188214302063),
 ('aquel', -0.1514982134103775),
 ('dijo', -0.15208400785923004),
 ('preciso', -0.1522463858127594),
 ('toda', -0.1555282026529312)]

In [293]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["océano"], topn=10)

[('extendía', 0.8936465978622437),
 ('perderse', 0.8889947533607483),
 ('extiende', 0.8862214088439941),
 ('escarpadas', 0.8834241628646851),
 ('pleno', 0.8718046545982361),
 ('lejanas', 0.8617182374000549),
 ('lago', 0.8612306118011475),
 ('llanura', 0.8576656579971313),
 ('arena', 0.8550131320953369),
 ('sombras', 0.8548770546913147)]

In [294]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["océano"])

[('tío—', -0.21445253491401672),
 ('señor', -0.21939441561698914),
 ('saknussemm', -0.23258419334888458),
 ('—¿qué', -0.23936977982521057),
 ('—me', -0.2497674822807312),
 ('hora', -0.2719522714614868),
 ('palabra', -0.2807517647743225),
 ('profesor—', -0.28218546509742737),
 ('—sí', -0.2872677743434906),
 ('—prosiguió', -0.2886728048324585)]

In [295]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["volcanes"], topn=10)

[('ciudades', 0.8842154145240784),
 ('subterráneos', 0.8790903687477112),
 ('volcánica', 0.8721573352813721),
 ('verdes', 0.8671268224716187),
 ('despojos', 0.8658183813095093),
 ('encuentran', 0.8641661405563354),
 ('blanca', 0.8639747500419617),
 ('fuegos', 0.8638566732406616),
 ('naturales', 0.8610393404960632),
 ('límites', 0.8604951500892639)]

In [296]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["volcanes"])

[('hora', -0.2583235800266266),
 ('hans', -0.3182982802391052),
 ('cabo', -0.3457009494304657),
 ('siguiente', -0.34586507081985474),
 ('momento', -0.3466376066207886),
 ('marcha', -0.3485548198223114),
 ('balsa', -0.35376596450805664),
 ('aún', -0.3585411310195923),
 ('horas', -0.3638328015804291),
 ('mañana', -0.3652820885181427)]

In [297]:
# el método `get_vector` permite obtener los vectores:
vector_terrestre = w2v_model.wv.get_vector("terrestre")
print(vector_terrestre)

[-0.11609855  0.41066664  0.12490825 -0.04590978 -0.00272756 -0.36417156
 -0.2121443  -0.06115238  0.1620348  -0.19342074 -0.04239405  0.10849806
  0.20152351 -0.09648021 -0.3744369  -0.14430569 -0.0236267  -0.01139157
  0.11207287  0.16732539 -0.19072188  0.24431227 -0.23543732  0.37626895
  0.46231255  0.08035368 -0.08366849  0.00573804  0.12255966 -0.01669434
 -0.25496033 -0.28685826  0.04598035 -0.16678777 -0.18414898  0.15583426
 -0.24265192 -0.40619272 -0.34751582 -0.06433387  0.1475789  -0.12580456
 -0.14835085 -0.06192503  0.15771057 -0.17873818 -0.00117047  0.43849513
 -0.2684275  -0.17496102  0.10811751 -0.19901453 -0.3125706  -0.00507751
  0.13223077 -0.02490409 -0.05138683 -0.05787152  0.02397081  0.26663625
 -0.16062982  0.02616561 -0.3599248   0.61994404  0.14273891 -0.04592519
 -0.18233116  0.01260114 -0.16452734  0.21656048 -0.09538787 -0.33547622
  0.18845129 -0.2249483   0.08292132  0.23494852 -0.38659692  0.05359088
  0.19062133  0.06405937 -0.3054527  -0.12051918 -0

In [298]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_terrestre)

[('terrestre', 0.9999999403953552),
 ('mineral', 0.8710821866989136),
 ('sinuosidades', 0.7647475600242615),
 ('gases', 0.7615578174591064),
 ('interiores', 0.7610321640968323),
 ('hulla', 0.7534886002540588),
 ('aumento', 0.7461691498756409),
 ('grietas', 0.7346856594085693),
 ('límite', 0.728818416595459),
 ('suponer', 0.7287749648094177)]

In [299]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["terrestre"], topn=10)

[('mineral', 0.8710821866989136),
 ('sinuosidades', 0.7647475600242615),
 ('gases', 0.7615578174591064),
 ('interiores', 0.7610321640968323),
 ('hulla', 0.7534886002540588),
 ('aumento', 0.7461691498756409),
 ('grietas', 0.7346855998039246),
 ('límite', 0.728818416595459),
 ('suponer', 0.728774905204773),
 ('base', 0.7269209027290344)]

### 5 - Visualizar agrupación de vectores

In [300]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [301]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [302]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [303]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# **Conclusiones**

**Texto**: "Viaje al centro de la Tierra"

**Preprocesamiento**: se eliminaron las palabras de tipo "stopwords" de los documentos, debido a que en un ensayo inicial, el modelo tenía un rendimiento bajo mostrando como palabras relacionadas algunas que parecían tener una relación semántica coherente.


---


### **Ensayo 1**

El modelo fue capaz de aprender vectores representativos que reflejan contexto semántico.

Las palabras consultadas ("terrestre", "roca", "volcanes") muestran relaciones con otras palabra en forma coherente.

Revisando las visualizaciones en 2D y 3D, se observa agrupaciones también coherentes.


- **Palabras ejemplo (Ensayo 1)**

| Palabra base | Palabras más similares                                                                                  | Puntajes de similitud     |
|--------------|----------------------------------------------------------------------------------------------------------|----------------------------|
| **terrestre** | través, macizo, aguas, capas, mineral, granito, temperatura, masa, radio, plantas                      | 0.875 → 0.769              |
| **roca**      | nubes, lava, cuyas, forma, halla, formado, ciertos, aspecto, cuyo, bóveda                              | 0.946 → 0.929              |
| **volcanes**  | alegría, escapó, geólogos, atlántico, límites, asombro, común, subterráneos, blanca, volcánica         | 0.991 → 0.990              |


- **Configuración de hiperparámetros**


```
epochs=20

w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300, # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram
```

---


### **Ensayo 2**
En este ensayo se aumento de vector_size=500 y epochs=40.

Los valores de similitud han disminuido respecto al Ensayo 1. En los gráficos, no se aprecian tan bien las agrupaciones de palanras.

Las palabras similares están menos relacionadas semánticamente (por ejemplo, “gusto” como similar a “volcanes” no es muy intuitivo).

El aumento de vector_size y epochs no mejoró el resultado, y posiblemente lo haya llevado a un sobreajuste.

- **Palabras ejemplo (Ensayo 2)**

| Palabra base | Palabras más similares                                                                                         | Puntajes de similitud     |
|--------------|-----------------------------------------------------------------------------------------------------------------|----------------------------|
| **terrestre** | mineral, gases, interiores, suponer, hulla, límite, sinuosidades, opinión, base, enfriamiento                 | 0.800 → 0.653              |
| **roca**      | sorprendente, atracción, pozo, trozo, convicción, dedo, marchaba, abierto, existía, criptograma               | 0.617 → 0.566              |
| **volcanes**  | ciudades, mares, gusto, naturales, volcánica, principal, árboles, subterráneos, encuentran, despojos          | 0.746 → 0.707              |


- **Configuración de hiperparámetros**


```
epochs=40

w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=500, # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram
```

### **Otros ensayos**

Disminuir el "vector_size", por ejemplo a 200 o menos también empeoró el rendimiento, disminuyendo los valores de similitud. Y en este caso, aumentar la cantidad de epochs tampoco ayudó.

Por otro lado aumentar el "window", para considerar más cantidad de palabras antes y después tuvo el mismo efecto, disminuyendo los valores de similitud.